## Preprocessing

In [141]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [156]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

columns_to_drop = ['EIN', 'NAME']
df_clean = application_df.drop(columns=columns_to_drop)

print(df_clean.head())

  APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  \
0   Association       1              0                      N     5000   
1  Co-operative       1         1-9999                      N   108590   
2   Association       1              0                      N     5000   
3         Trust       1    10000-24999                      N     6692   
4         Trust       1  100000-499999                      N   142590   

   IS_SUCCESSFUL  
0              1  
1              1  
2              0  
3              1  
4              1  


In [157]:
# Determine the number of unique values in each column.
unique_value_counts = df_clean.nunique()
print(unique_value_counts)


APPLICATION_TYPE            10
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [158]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = df_clean['APPLICATION_TYPE'].value_counts()
print(application_type_counts)


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64


In [169]:
# Choose a cutoff value and create a list of application types to be replaced

cutoff_value = 200

# Calculate value counts of application types
application_type_counts = df_clean['APPLICATION_TYPE'].value_counts()

# Create a list of application types to be replaced
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in the dataframe
for app in application_types_to_replace:
    df_clean['APPLICATION_TYPE'] = df_clean['APPLICATION_TYPE'].replace(app, "Other")

# Check the value counts after binning
print(df_clean['APPLICATION_TYPE'].value_counts())

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
# Ensure code works
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [170]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
classification_counts = df_clean['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64


In [171]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
classification_counts = df_clean['CLASSIFICATION'].value_counts()
classification_counts_filtered = classification_counts[classification_counts > 1]
print(classification_counts_filtered)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64


In [173]:
cutoff_value= 1000

# Calculate value counts of classifications
classification_counts = df_clean['CLASSIFICATION'].value_counts()

# Create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

for cls in classifications_to_replace:
    df_clean['CLASSIFICATION'] = df_clean['CLASSIFICATION'].replace(cls, "Other")

# Check the value counts
print(df_clean['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [174]:
# Convert categorical data to numeric with `pd.get_dummies`

df_encoded = pd.get_dummies(df_clean)
print(df_encoded.head())

   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                       0   
1       1   108590              1                       0   
2       1     5000              0                       0   
3       1     6692              1                       0   
4       1   142590              1                       0   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                     1                     0                    0   
1                     0                     0                    1   
2                     0                     0                    0   
3                     0                     0                    1   
4                     0                     0                    1   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                    0                    0                    0  ...   
1                    0                    0                    0  

In [175]:
# Split our preprocessed data into our features and target arrays
X = df_encoded.drop(columns=['IS_SUCCESSFUL'])
y = df_encoded['IS_SUCCESSFUL'].values

# Print the shapes of the arrays to verify
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
#  YOUR CODE GOES HERE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the datasets to verify
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X: (34299, 43)
Shape of y: (34299,)
Shape of X_train: (27439, 43)
Shape of X_test: (6860, 43)
Shape of y_train: (27439,)
Shape of y_test: (6860,)


In [176]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [180]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
from tensorflow.keras.layers import Dense
nn = tf.keras.models.Sequential()
layer1 = 80
layer2 = 30
# Add input layer
nn.add(Dense(units= layer1, activation='relu', input_dim=X_train.shape[1]))

# Add hidden layers
nn.add(Dense(units=layer2, activation='relu'))

# Add output layer
nn.add(Dense(units=1, activation='sigmoid'))  # Adjust activation function based on your task

# Print a summary of the model architecture
nn.summary()


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 80)                3520      
                                                                 
 dense_45 (Dense)            (None, 30)                2430      
                                                                 
 dense_46 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [186]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam',metrics = ['accuracy'])  # Adjust loss and optimizer based on your task

In [187]:
# Train the model
history = nn.fit(X_train, y_train, epochs=20, callbacks=[early_stopping])

Epoch 1/20
855/858 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5321

858/858 [==============================] - 3s 2ms/step - loss: 0.6913 - accuracy: 0.5320
Epoch 2/20
845/858 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5325

858/858 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 3/20
839/858 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5322

858/858 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 4/20
845/858 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5320

858/858 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 5/20
856/858 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5319

...

858/858 [==============================] - 3s 4ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 17/20
852/858 [============================>.] - ETA: 0s - loss: 0.6915 - accuracy: 0.5316

858/858 [==============================] - 2s 3ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 18/20
841/858 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5309

858/858 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5306
Epoch 19/20
857/858 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5320

858/858 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5320
Epoch 20/20
858/858 [==============================] - ETA: 0s - loss: 0.6914 - accuracy: 0.5320

858/858 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5320


In [188]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.8145 - accuracy: 0.5700 - 394ms/epoch - 2ms/step
Loss: 0.814509928226471, Accuracy: 0.5699708461761475


In [189]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save('module21challenge.h5')